# Introduction to Deep Learning 67822 - [Ex1](https://docs.google.com/document/d/11Q1ejfwTH_tHjdQob0gYLA3bS88lNsBStpBWz085rB0/edit?tab=t.0)

## Section 1: Load and Prepare the Data

### Split to train and test sets randomly at a 1:9 ratio.

```bash
[Raw Files]
    |
    |---> Positive Files (e.g. A0207_pos.txt, etc.)
    |---> Negative File (e.g. negs.txt)
    |
    v
[load_and_split_data()]
    |
    |---> Read & shuffle positive peptides
    |---> Read & shuffle negative peptides (per allele)
    |---> Split each into Train/Test (90%/10%)
    |---> Combine pos+neg per allele
    |
    v
[Datasets Dict]
{ "A0207": (train_data, test_data), ... }
    |
    |  - train_data: List of (peptide_str, allele, label)
    |  - test_data:  List of (peptide_str, allele, label)
    |
    v
[summarize_datasets()]
    |
    |---> Count and print stats:
    |     - Total sizes, % train/test
    |     - % positive/negative in each set
    |
    v
[prepare_data()]
    |
    |---> peptide_to_indices(): Convert each peptide string → list of indices
    |---> Convert (X, y) to PyTorch tensors for each allele
    |
    v
[Processed Datasets Dict]
{ "A0207": (X_train, y_train, X_test, y_test), ... }
    |
    |  - X_train: Tensor of shape (n_samples, 9)
    |  - y_train: Tensor of shape (n_samples,)
    |
    v
[summarize_processed_datasets()]
    |
    |---> Show tensor shapes
    |---> Show one example input+label
    |
    v
[Ready for Training]
```

In [1]:
from dataset import load_and_split_data

# Load and split data
datasets = load_and_split_data()

Dataset Summary:
  Allele  Train Size  Test Size Train (%) Test (%) Train Positive (%)  \
0  A0207       25024       2782    89.99%   10.01%             11.92%   
1  A0101       23184       2577    90.00%   10.00%              4.93%   
2  A2402       24028       2671    90.00%   10.00%              8.27%   
3  A0201       24394       2712    89.99%   10.01%              9.64%   
4  A0203       23687       2633    90.00%   10.00%              6.94%   
5  A0301       23535       2616    90.00%   10.00%              6.34%   

  Train Negative (%) Test Positive (%) Test Negative (%)  
0             88.08%            11.93%            88.07%  
1             95.07%             4.93%            95.07%  
2             91.73%             8.27%            91.73%  
3             90.36%             9.66%            90.34%  
4             93.06%             6.95%            93.05%  
5             93.66%             6.35%            93.65%  


In [2]:
from dataset import prepare_data

# Prepare tensor datasets for all alleles
# Our datasets variable is already in the format {allele: (train_data, test_data)}
processed_datasets = prepare_data(datasets)
# Create a dictionary to store alleles and their data in a more organized way
allele_data = {}
for allele, (X_train_allele, y_train_allele, X_test_allele, y_test_allele) in processed_datasets.items():
    allele_data[allele] = {
        'X_train': X_train_allele,
        'y_train': y_train_allele,
        'X_test': X_test_allele,
        'y_test': y_test_allele
    }

Processed Dataset Summary:
  Allele X_train shape y_train shape X_test shape y_test shape  \
0  A0207    (25024, 9)      (25024,)    (2782, 9)      (2782,)   
1  A0101    (23184, 9)      (23184,)    (2577, 9)      (2577,)   
2  A2402    (24028, 9)      (24028,)    (2671, 9)      (2671,)   
3  A0201    (24394, 9)      (24394,)    (2712, 9)      (2712,)   
4  A0203    (23687, 9)      (23687,)    (2633, 9)      (2633,)   
5  A0301    (23535, 9)      (23535,)    (2616, 9)      (2616,)   

             Example input (indices)  Example label (binary)  
0    [7, 10, 8, 8, 7, 14, 3, 15, 19]                       0  
1     [5, 16, 10, 14, 5, 7, 7, 0, 8]                       0  
2    [14, 16, 7, 4, 12, 9, 4, 10, 8]                       0  
3    [18, 9, 2, 15, 17, 6, 13, 9, 9]                       0  
4  [16, 9, 19, 3, 10, 8, 15, 13, 17]                       0  
5       [4, 7, 2, 2, 7, 3, 9, 10, 4]                       0  


## Section 2 

### a.

#### How would you represent these 9-mers of amino acids?

Each amino acid in the 9-mer is represented using an integer index (from a fixed map of 20 amino acids).

Each peptide becomes a list of 9 integers → converted to a tensor of shape (9,).

During training, this tensor goes through an embedding layer to turn it into a (9, emb_dim) representation.

A batch of these peptides becomes a (B, 9) tensor → embedded to (B, 9, emb_dim).

```bash
"A R G D E K Y I L"   ← Peptide string (length = 9 amino acids)
       |
       v
[peptide_to_indices()]
       |
       v
[2, 17, 5, 3, 4, 8, 18, 6, 11]   ← Each amino acid mapped to index using AA_TO_IDX
       |
       v
[PyTorch Tensor of shape (9,)]   ← Final input format to the model
```

Then, during model training:


```bash
(Batch of peptides)        ← Shape: (B, 9)
       |
       v
nn.Embedding(20, emb_dim)  ← Shape: (B, 9) → (B, 9, emb_dim)
```


#### How would you represent the associate alleles?

```bash
"Allele Name" (e.g. "A0207")
       |
       v
[ALLELE_LABEL_MAP]  --> Maps to Integer Label
       |
       v
Class ID (int)  → e.g. 4
       |
       v
Used as y-label in training:
    - In binary models: y = 1 if peptide belongs to this allele, else y = 0
    - In multiclass model: y = integer from 0–6 (NEG = 0)
       |
       v
During inference:
[MultiAlleleEnsemble]
       |
       v
Each allele has its own slot in fixed order:
['A0101', 'A0201', 'A0203', 'A0207', 'A0301', 'A2402']
       |
       v
Model output → tensor of shape (B, 6)
       |
       v
Each column corresponds to confidence for that allele
e.g. output[0] = [0.01, 0.03, 0.85, 0.92, 0.02, 0.07]
                     ↑     ↑     ↑     ↑     ↑     ↑
                   A0101 A0201 A0203 A0207 A0301 A2402
```

### b.

#### What will the network’s input dimension be under this representation choice?

```bash
Peptide (9-mer string)  
    →  e.g. "ARDKQWLTV"
        |
        v
[peptide_to_indices()] using AA_TO_IDX
    →  [2, 17, 3, 8, 12, 15, 10, 13, 18]      (length = 9)
        |
        v
Tensor of shape (9,)                        (indices of amino acids)
        |
        v
nn.Embedding(NUM_AMINO_ACIDS=20, EMB_DIM=d)
    → Each index → vector of size EMB_DIM
    → Output shape: (9, EMB_DIM)
        |
        v
nn.Flatten()
    → Flattens (9, EMB_DIM) → (9 * EMB_DIM,)
        |
        v
Final Input Dimension = 9 * EMB_DIM
```


#### Implement an MLP that keeps this dimension for 2 inner layers

In [3]:
# Section 2B: Simple MLP with 2 equal-sized hidden layers
from model import PeptideToHLAClassifier_2B, MultiAlleleEnsemble
from training import create_data_loaders, setup_training, train_model
from hyperparameters import allele_data_hyperparameters

# Create simplified hyperparameters dictionary by removing advanced features
simplified_hyperparameters = {}
for allele in allele_data_hyperparameters:
    # Start with the basic hyperparameters
    simplified_hyperparameters[allele] = {
        'EMB_DIM': allele_data_hyperparameters[allele]['EMB_DIM'],
        'FC_HIDDEN_DIM': allele_data_hyperparameters[allele]['FC_HIDDEN_DIM'],
        'loss_function': 'BCEWithLogitsLoss',
        'BATCH_SIZE': allele_data_hyperparameters[allele]['BATCH_SIZE'],
        'LEARNING_RATE': allele_data_hyperparameters[allele]['LEARNING_RATE'],
        'EPOCHS': 10,  # Fixed number of epochs for all models
        'THRESHOLD': 0.5,  # Fixed threshold
        'PATIENCE': 999,  # Effectively disable early stopping with a very large patience
    }

print("# Training simple 2B architecture with two equal-sized hidden layers")
print("# Using existing training infrastructure with simplified settings:")
print("# - Fixed 10 epochs for all models")
print("# - No effective early stopping (patience=999)")
print("# - Fixed threshold (0.5)")
print("# - No dropout")
print("# - Simple architecture (2 equal hidden layers)")

# Dictionary to store results for plotting
to_plot_2B = {}

# Dictionary to store trained models
Mini_Nets_2B = {}

for selected_allele in allele_data.keys():
    print(f"\n--- Processing {selected_allele} Dataset with simple 2B architecture ---")
    
    # Create model instance - simple architecture with 2 equal-sized hidden layers
    model = PeptideToHLAClassifier_2B(
        emb_dim=simplified_hyperparameters[selected_allele]['EMB_DIM'],
        fc_hidden_dim=simplified_hyperparameters[selected_allele]['FC_HIDDEN_DIM'],
    )

    # Create data loaders using your existing function
    train_loader, test_loader = create_data_loaders(
        X_train=allele_data[selected_allele]['X_train'],
        y_train=allele_data[selected_allele]['y_train'],
        X_test=allele_data[selected_allele]['X_test'],
        y_test=allele_data[selected_allele]['y_test']
    )

    # Setup loss function and optimizer using your existing function
    loss_fn, optimizer = setup_training(
        model=model,
        loss_function=simplified_hyperparameters[selected_allele]['loss_function'],
        learning_rate=simplified_hyperparameters[selected_allele]['LEARNING_RATE'],
        y_train=allele_data[selected_allele]['y_train']
    )

    # Print hyperparameters
    print(
        f"emb_dim: {simplified_hyperparameters[selected_allele]['EMB_DIM']}, "
        f"fc_hidden_dim: {simplified_hyperparameters[selected_allele]['FC_HIDDEN_DIM']}, "
        f"epochs: {simplified_hyperparameters[selected_allele]['EPOCHS']}, "
        f"batch_size: {simplified_hyperparameters[selected_allele]['BATCH_SIZE']}, "
        f"loss_function: {simplified_hyperparameters[selected_allele]['loss_function']}, "
        f"learning_rate: {simplified_hyperparameters[selected_allele]['LEARNING_RATE']}, "
        f"threshold: {simplified_hyperparameters[selected_allele]['THRESHOLD']}, "
        f"patience: {simplified_hyperparameters[selected_allele]['PATIENCE']}, "
        f"train_size: {len(allele_data[selected_allele]['X_train'])}, "
        f"test_size: {len(allele_data[selected_allele]['X_test'])}, "
        f"optimizer: {optimizer.__class__.__name__}"
    )

    # Train the model using your existing train_model function
    # But with simplified settings that effectively disable early stopping
    train_losses, test_losses, accuracies, pos_accuracies, neg_accuracies, final_outputs = train_model(
        model=model,
        train_loader=train_loader,
        test_loader=test_loader,
        loss_fn=loss_fn,
        optimizer=optimizer,
        epochs=simplified_hyperparameters[selected_allele]['EPOCHS'],
        threshold=simplified_hyperparameters[selected_allele]['THRESHOLD'],
        patience=simplified_hyperparameters[selected_allele]['PATIENCE'],  # Very large patience = no early stopping
        return_final_outputs=True,
    )

    # Store results for plotting
    to_plot_2B[selected_allele] = {
        'train_losses': train_losses,
        'test_losses': test_losses,
        'accuracies': accuracies,
        'pos_accuracies': pos_accuracies,
        'neg_accuracies': neg_accuracies,
        'epochs': len(train_losses),
        'raw_output': final_outputs,
    }
    
    # Print some statistics about the raw outputs
    print(f"\n{selected_allele} – shape of raw outputs: {final_outputs.shape}")
    print(f"{selected_allele} – first ten raw neuron values:\n{final_outputs[:10].numpy()}\n")
    
    # Store the trained model
    Mini_Nets_2B[selected_allele] = model

# Create the ensemble model with all trained submodels
Model2B = MultiAlleleEnsemble(Mini_Nets_2B)
print("\nModel 2B training complete. Created ensemble model combining all allele-specific models.")

# Training simple 2B architecture with two equal-sized hidden layers
# Using existing training infrastructure with simplified settings:
# - Fixed 10 epochs for all models
# - No effective early stopping (patience=999)
# - Fixed threshold (0.5)
# - No dropout
# - Simple architecture (2 equal hidden layers)

--- Processing A0207 Dataset with simple 2B architecture ---
emb_dim: 32, fc_hidden_dim: 128, epochs: 10, batch_size: 64, loss_function: BCEWithLogitsLoss, learning_rate: 0.0005, threshold: 0.5, patience: 999, train_size: 25024, test_size: 2782, optimizer: Adam
Epoch  1/10 | Train Loss: 0.6516 | Test Loss: 0.6089 | Overall Accuracy: 84.65% | Pos Accuracy: 89.76% | Neg Accuracy: 83.96% | LR: 0.000500
Epoch  2/10 | Train Loss: 0.5373 | Test Loss: 0.5802 | Overall Accuracy: 84.94% | Pos Accuracy: 90.66% | Neg Accuracy: 84.16% | LR: 0.000500
Epoch  3/10 | Train Loss: 0.5022 | Test Loss: 0.5879 | Overall Accuracy: 81.49% | Pos Accuracy: 93.07% | Neg Accuracy: 79.92% | LR: 0.000500
Epoch  

#### plot the resulting train and test losses.

In [ ]:
from evaluation import plot_training_results
# Generate plots for each allele
for selected_allele, data in to_plot_2B.items():
    plots = plot_training_results(
        selected_allele, 
        data['train_losses'], 
        data['test_losses'], 
        data['accuracies'], 
        data['pos_accuracies'], 
        data['neg_accuracies'], 
        data['epochs']
    )

#### Does the input dimension cause training problems?
The model quickly begins to overfit, shown by the continuously decreasing training loss while test loss begins to increase after a few epochs (particularly visible in alleles like A0201 and A0203)
it tends to favor the majority negative class, causing positive accuracy to drop significantly in later epochs, and the raw output values become extremely large (-40 to +8), indicating the model is becoming overconfident.
when the learning rate decreases due to the scheduler, we often see a spike in test loss, suggesting that maybe the model is stuck in a poor local minimum.

### c.

#### Describe a network architecture that avoids the problems seen

```bash
[Peptide Tensor Data (X_train/y_train)]
      |
      v
[create_data_loaders()] ───────────────> PyTorch DataLoader (batched input)
      |
      v
[PeptideToHLAClassifier_2C]
    |
    |---> Embedding: (B, 9) → (B, 9, emb_dim)
    |---> Flatten: (B, 9, emb_dim) → (B, 9 * emb_dim)
    |---> FC1: Linear → BN → LeakyReLU → Dropout
    |---> FC2: Linear → BN → LeakyReLU → Dropout
    |---> FC3: Linear → BN → LeakyReLU → Dropout
    |---> Output Layer: Linear → raw logits (B)
    |
    v
[setup_training()]
    |
    |---> Loss: BCEWithLogitsLoss (with class weighting)
    |---> Optimizer: Adam
    |
    v
[train_model()]
    |
    |---> Epoch loop:
    |      - Forward pass
    |      - Loss backward + optimizer.step()
    |      - Validation
    |      - Accuracy (overall, pos, neg)
    |      - Scheduler & EarlyStopping
    |
    v
[Save Model]
    |
    v
save_model(model, allele)

Repeat for all 6 alleles:

            ┌────────────┐
            │   A0101    │
            │   A0201    │
            │   A0203    │
            │   A0207    │
            │   A0301    │
            │   A2402    │
            └────┬───────┘
                 |
                 v
[load_allele_models()] → Builds:
[MultiAlleleEnsemble]
    |
    |---> Forwards peptide to each of the 6 trained models
    |---> Collects outputs → tensor of shape (B, 6)
    |
    v
[predict_allele()]
    |
    |---> Converts peptide → tensor
    |---> Runs through MultiAlleleEnsemble
    |---> Applies thresholds
    |---> Returns: (predicted_allele, confidence)
```


In [5]:
from model import PeptideToHLAClassifier_2C
from training import create_data_loaders, setup_training, train_model
from hyperparameters import allele_data_hyperparameters

to_plot_2C = {}

Mini_Nets_2C = {}
for selected_allele in allele_data.keys():
    print(f"--- Processing {selected_allele} Dataset ---")
    
    # Create model instance with configurable dropout
    model = PeptideToHLAClassifier_2C(
        emb_dim=allele_data_hyperparameters[selected_allele]['EMB_DIM'],
        fc_hidden_dim=allele_data_hyperparameters[selected_allele]['FC_HIDDEN_DIM'],
        dropout_rates=allele_data_hyperparameters[selected_allele].get('DROPOUT_RATES', [0.3, 0.2, 0.1])
    )

    # Create data loaders
    train_loader, test_loader = create_data_loaders(
        X_train = allele_data[selected_allele]['X_train'],
        y_train = allele_data[selected_allele]['y_train'],
        X_test = allele_data[selected_allele]['X_test'],
        y_test = allele_data[selected_allele]['y_test']
    )

    # Setup loss function and optimizer
    loss_fn, optimizer = setup_training(
        model=model,
        loss_function=allele_data_hyperparameters[selected_allele]['loss_function'],
        learning_rate=allele_data_hyperparameters[selected_allele]['LEARNING_RATE'],
        y_train=allele_data[selected_allele]['y_train']
    )

    # Print hyperparameters
    print(
        f"emb_dim: {allele_data_hyperparameters[selected_allele]['EMB_DIM']}, "
        f"fc_hidden_dim: {allele_data_hyperparameters[selected_allele]['FC_HIDDEN_DIM']}, "
        f"epochs: {allele_data_hyperparameters[selected_allele]['EPOCHS']}, "
        f"batch_size: {allele_data_hyperparameters[selected_allele]['BATCH_SIZE']}, "
        f"loss_function: {allele_data_hyperparameters[selected_allele]['loss_function']}, "
        f"learning_rate: {allele_data_hyperparameters[selected_allele]['LEARNING_RATE']}, "
        f"threshold: {allele_data_hyperparameters[selected_allele]['THRESHOLD']}, "
        f"patience: {allele_data_hyperparameters[selected_allele].get('PATIENCE', 5)}, "
        f"dropout_rates: {allele_data_hyperparameters[selected_allele].get('DROPOUT_RATES', [0.3, 0.2, 0.1])}, "
        f"train_size: {len(allele_data[selected_allele]['X_train'])}, "
        f"test_size: {len(allele_data[selected_allele]['X_test'])}, "
        f"optimizer: {optimizer.__class__.__name__}"
    )

    train_losses, test_losses, accuracies, pos_accuracies, neg_accuracies, final_outputs = train_model(
        model=model,
        train_loader=train_loader,
        test_loader=test_loader,
        loss_fn=loss_fn,
        optimizer=optimizer,
        epochs=allele_data_hyperparameters[selected_allele]['EPOCHS'],
        threshold=allele_data_hyperparameters[selected_allele]['THRESHOLD'],
        patience=allele_data_hyperparameters[selected_allele].get('PATIENCE', 5),
        return_final_outputs=True,
    )

    to_plot_2C[selected_allele] = {
        'train_losses': train_losses,
        'test_losses': test_losses,
        'accuracies': accuracies,
        'pos_accuracies': pos_accuracies,
        'neg_accuracies': neg_accuracies,
        'epochs': len(train_losses),
        'raw_output': final_outputs,
    }
    
    print(f"{selected_allele} – shape of raw outputs:", final_outputs.shape)
    print(f"{selected_allele} – first ten raw neuron values\n {final_outputs[:10].numpy()}\n")
    Mini_Nets_2C[selected_allele] = model
Model2C = MultiAlleleEnsemble(Mini_Nets_2C)

--- Processing A0207 Dataset ---
emb_dim: 32, fc_hidden_dim: 128, epochs: 15, batch_size: 64, loss_function: BCEWithLogitsLoss, learning_rate: 0.0005, threshold: 0.52, patience: 4, dropout_rates: [0.3, 0.2, 0.1], train_size: 25024, test_size: 2782, optimizer: Adam
Epoch  1/15 | Train Loss: 0.7068 | Test Loss: 0.6115 | Overall Accuracy: 81.70% | Pos Accuracy: 91.27% | Neg Accuracy: 80.41% | LR: 0.000500
Epoch  2/15 | Train Loss: 0.5696 | Test Loss: 0.5777 | Overall Accuracy: 81.92% | Pos Accuracy: 94.88% | Neg Accuracy: 80.16% | LR: 0.000500
Epoch  3/15 | Train Loss: 0.5396 | Test Loss: 0.5690 | Overall Accuracy: 82.71% | Pos Accuracy: 93.98% | Neg Accuracy: 81.18% | LR: 0.000500
Epoch  4/15 | Train Loss: 0.5161 | Test Loss: 0.5737 | Overall Accuracy: 82.49% | Pos Accuracy: 93.67% | Neg Accuracy: 80.98% | LR: 0.000500
Epoch  5/15 | Train Loss: 0.4949 | Test Loss: 0.5685 | Overall Accuracy: 83.47% | Pos Accuracy: 93.07% | Neg Accuracy: 82.16% | LR: 0.000500
Epoch  6/15 | Train Loss: 0.49

#### explain all your design choices.

TODO - Explaining the code and the results

#### Plot the train / tests losses. Include the plots obtained in your report.

In [ ]:

from evaluation import plot_training_results

# Generate plots
for selected_allele, data in to_plot_2C.items():
    train_losses = data['train_losses']
    test_losses = data['test_losses']
    accuracies = data['accuracies']
    pos_accuracies = data['pos_accuracies']
    neg_accuracies = data['neg_accuracies']
    epochs = data['epochs']
    plot_training_results(selected_allele, train_losses, test_losses, 
                         accuracies, pos_accuracies, neg_accuracies, epochs)

### d.

#### Remove the non-linear operators from your network

#### report the results obtained.

In [7]:
from model import PeptideToHLAClassifier_2D, MultiAlleleEnsemble
from training import create_data_loaders, setup_training, train_model
from hyperparameters import allele_data_hyperparameters

to_plot_2D = {}

Mini_Nets_2D = {}
for selected_allele in allele_data.keys():
    print(f"--- Processing {selected_allele} Dataset ---")
    
    # Create model instance with configurable dropout
    model = PeptideToHLAClassifier_2D(
        emb_dim=allele_data_hyperparameters[selected_allele]['EMB_DIM'],
        fc_hidden_dim=allele_data_hyperparameters[selected_allele]['FC_HIDDEN_DIM'],
        dropout_rates=allele_data_hyperparameters[selected_allele].get('DROPOUT_RATES', [0.3, 0.2, 0.1])
    )

    # Create data loaders
    train_loader, test_loader = create_data_loaders(
        X_train = allele_data[selected_allele]['X_train'],
        y_train = allele_data[selected_allele]['y_train'],
        X_test = allele_data[selected_allele]['X_test'],
        y_test = allele_data[selected_allele]['y_test']
    )

    # Setup loss function and optimizer
    loss_fn, optimizer = setup_training(
        model=model,
        loss_function=allele_data_hyperparameters[selected_allele]['loss_function'],
        learning_rate=allele_data_hyperparameters[selected_allele]['LEARNING_RATE'],
        y_train=allele_data[selected_allele]['y_train']
    )

    # Print hyperparameters
    print(
        f"emb_dim: {allele_data_hyperparameters[selected_allele]['EMB_DIM']}, "
        f"fc_hidden_dim: {allele_data_hyperparameters[selected_allele]['FC_HIDDEN_DIM']}, "
        f"epochs: {allele_data_hyperparameters[selected_allele]['EPOCHS']}, "
        f"batch_size: {allele_data_hyperparameters[selected_allele]['BATCH_SIZE']}, "
        f"loss_function: {allele_data_hyperparameters[selected_allele]['loss_function']}, "
        f"learning_rate: {allele_data_hyperparameters[selected_allele]['LEARNING_RATE']}, "
        f"threshold: {allele_data_hyperparameters[selected_allele]['THRESHOLD']}, "
        f"patience: {allele_data_hyperparameters[selected_allele].get('PATIENCE', 5)}, "
        f"dropout_rates: {allele_data_hyperparameters[selected_allele].get('DROPOUT_RATES', [0.3, 0.2, 0.1])}, "
        f"train_size: {len(allele_data[selected_allele]['X_train'])}, "
        f"test_size: {len(allele_data[selected_allele]['X_test'])}, "
        f"optimizer: {optimizer.__class__.__name__}"
    )

    train_losses, test_losses, accuracies, pos_accuracies, neg_accuracies, final_outputs = train_model(
        model=model,
        train_loader=train_loader,
        test_loader=test_loader,
        loss_fn=loss_fn,
        optimizer=optimizer,
        epochs=allele_data_hyperparameters[selected_allele]['EPOCHS'],
        threshold=allele_data_hyperparameters[selected_allele]['THRESHOLD'],
        patience=allele_data_hyperparameters[selected_allele].get('PATIENCE', 5),
        return_final_outputs=True,
    )

    to_plot_2D[selected_allele] = {
        'train_losses': train_losses,
        'test_losses': test_losses,
        'accuracies': accuracies,
        'pos_accuracies': pos_accuracies,
        'neg_accuracies': neg_accuracies,
        'epochs': len(train_losses),
        'raw_output': final_outputs,
    }
    
    print(f"{selected_allele} – shape of raw outputs:", final_outputs.shape)
    print(f"{selected_allele} – first ten raw neuron values\n {final_outputs[:10].numpy()}\n")
    Mini_Nets_2D[selected_allele] = model
Model2D = MultiAlleleEnsemble(Mini_Nets_2D)

--- Processing A0207 Dataset ---
emb_dim: 32, fc_hidden_dim: 128, epochs: 15, batch_size: 64, loss_function: BCEWithLogitsLoss, learning_rate: 0.0005, threshold: 0.52, patience: 4, dropout_rates: [0.3, 0.2, 0.1], train_size: 25024, test_size: 2782, optimizer: Adam
Epoch  1/15 | Train Loss: 0.6400 | Test Loss: 0.6172 | Overall Accuracy: 82.71% | Pos Accuracy: 92.77% | Neg Accuracy: 81.35% | LR: 0.000500
Epoch  2/15 | Train Loss: 0.5838 | Test Loss: 0.6227 | Overall Accuracy: 85.37% | Pos Accuracy: 88.55% | Neg Accuracy: 84.94% | LR: 0.000500
Epoch  3/15 | Train Loss: 0.5709 | Test Loss: 0.6097 | Overall Accuracy: 84.65% | Pos Accuracy: 89.76% | Neg Accuracy: 83.96% | LR: 0.000500
Epoch  4/15 | Train Loss: 0.5676 | Test Loss: 0.5933 | Overall Accuracy: 83.61% | Pos Accuracy: 92.47% | Neg Accuracy: 82.41% | LR: 0.000500
Epoch  5/15 | Train Loss: 0.5600 | Test Loss: 0.6006 | Overall Accuracy: 84.40% | Pos Accuracy: 90.06% | Neg Accuracy: 83.63% | LR: 0.000500
Epoch  6/15 | Train Loss: 0.55

#### How do the results compare?

This function shows the expected results when removing non-linear operators: there is reduced expressivity, without non-linearities, the model is essentially a single linear transformation (composition of linear functions is linear), so it can't learn complex decision boundaries needed for peptide classification
    
i also see lower training capacity as the model underfits the data, showing higher training and test loss. also ther is worse generalization,because performance on test data drops significantly, with accuracy decreasing by 5-15% depending on the allele.
    
there is faster overfitting, With fewer parameters, the model begins overfitting earlier but to a less extreme degree, and also - class imbalance issues: The linear model struggles more with class imbalance, often showing very poor performance on the minority class.

For all alleles, we see a consistent pattern of decreased performance when removing the non-linear components from the network. This demonstrates that the non-linearities are essential for learning the complex patterns that distinguish binding from non-binding peptides across different HLA alleles.

### e.

#### Use your model to predict the detection of 9-amino-acids peptides from the Spike protein of the SARS-CoV-2

In [8]:
from predict import predict_allele
from collections import defaultdict, Counter
def get_spike_protein():
    with open("Data/spike.txt") as f:
        spike_sequence = f.read().strip().replace("\n", "").replace(" ", "")
    return spike_sequence

def extract_9mers(sequence):
    """Extract all possible 9-mer peptides from a protein sequence."""
    peptides = []
    positions = []
    
    for i in range(len(sequence) - 8):
        peptide = sequence[i:i+9]
        # Only include peptides with standard amino acids
        if all(aa in "ACDEFGHIKLMNPQRSTVWY" for aa in peptide):
            peptides.append(peptide)
            positions.append(i+1)  # 1-indexed position
            
    return peptides, positions

def predict_spike_peptides(model):
    """
    Predict binding of SARS-CoV-2 spike protein peptides to HLA alleles.
    
    Args:
        model: Trained MultiAlleleEnsemble model
        
    Returns:
        Dictionary mapping allele to list of (peptide, position, score) tuples
    """
    # Get the spike protein sequence
    spike_protein = get_spike_protein()
    print(f"SARS-CoV-2 Spike protein length: {len(spike_protein)} amino acids")
    
    # Extract all possible 9-mer peptides
    peptides, positions = extract_9mers(spike_protein)
    print(f"Total number of 9-mer peptides: {len(peptides)}")
    
    # Store results by allele
    results = defaultdict(list)
    
    # Process each peptide
    print("Predicting binding for each peptide...")
    for i, peptide in enumerate(peptides):
        # Use the provided predict_allele function
        predicted_allele, confidence = predict_allele(model, peptide)
        
        # Store result if binding is predicted
        if predicted_allele is not None:
            results[predicted_allele].append({
                'peptide': peptide,
                'position': positions[i],
                'score': confidence
            })
            
        # Progress indicator (every 100 peptides)
        if (i+1) % 100 == 0:
            print(f"Processed {i+1}/{len(peptides)} peptides...")
    
    # Sort results by score for each allele
    for allele in results:
        results[allele].sort(key=lambda x: x['score'], reverse=True)

    return results

def analyze_results(results):
    """
    Analyze and display prediction results.
    
    Args:
        results: Dictionary mapping allele to list of binding peptides
    """
    print("\n===== SARS-CoV-2 Spike Protein Binding Predictions =====")
    
    # Display top predictions per allele
    print("\nTop 3 binding peptides per allele:")
    for allele in sorted(results.keys()):
        print(f"\n{allele}:")
        
        if not results[allele]:
            print("  No binding peptides predicted")
            continue
            
        # Show top 3 peptides for each allele
        for i, result in enumerate(results[allele][:3]):
            print(f"  {i+1}. {result['peptide']} (Position: {result['position']}, Score: {result['score']:.4f})")
    
    # Find peptides that bind to multiple alleles
    all_peptides = []
    for peptides in results.values():
        all_peptides.extend([p['peptide'] for p in peptides])
    
    peptide_counts = Counter(all_peptides)
    multi_binders = [peptide for peptide, count in peptide_counts.items() if count > 1]
    
    if multi_binders:
        print("\nPeptides predicted to bind to multiple HLA alleles:")
        
        multi_binding_details = []
        
        for peptide in sorted(multi_binders):
            alleles_bound = []
            position = None
            
            for allele, peptides in results.items():
                for p in peptides:
                    if p['peptide'] == peptide:
                        alleles_bound.append((allele, p['score']))
                        position = p['position']
            
            # Sort by binding score
            alleles_bound.sort(key=lambda x: x[1], reverse=True)
            allele_str = ', '.join([f"{a} ({s:.4f})" for a, s in alleles_bound])
            
            multi_binding_details.append({
                'peptide': peptide,
                'position': position,
                'alleles': alleles_bound,
                'num_alleles': len(alleles_bound)
            })
            
            print(f"  {peptide} (Position: {position}) binds to: {allele_str}")
        
        # Sort by number of alleles bound and highest score
        multi_binding_details.sort(key=lambda x: (x['num_alleles'], x['alleles'][0][1]), reverse=True)
        
        print("\nTop 3 most promising vaccine candidates (binding to most alleles):")
        for i, detail in enumerate(multi_binding_details[:3]):
            allele_str = ', '.join([f"{a} ({s:.4f})" for a, s in detail['alleles']])
            print(f"  {i+1}. {detail['peptide']} (Position: {detail['position']}) binds to: {allele_str}")



# In a real implementation, you would load your trained model here

print("Loading trained model...")
model = Model2C
        
# Run predictions on spike protein peptides
results = predict_spike_peptides(model)

# Analyze and display results
analyze_results(results)


Loading trained model...
SARS-CoV-2 Spike protein length: 1273 amino acids
Total number of 9-mer peptides: 1265
Predicting binding for each peptide...
Peptide: MFVFLVLLP
  A0101: 0.0411 (threshold: 0.5) ✗
  A0201: 0.4978 (threshold: 0.45) ✓
  A0203: 0.3239 (threshold: 0.5) ✗
  A0207: 0.2690 (threshold: 0.52) ✗
  A0301: 0.7978 (threshold: 0.4) ✓
  A2402: 0.8908 (threshold: 0.55) ✓
Peptide: FVFLVLLPL
  A0101: 0.2194 (threshold: 0.5) ✗
  A0201: 0.7660 (threshold: 0.45) ✓
  A0203: 0.5204 (threshold: 0.5) ✓
  A0207: 0.7324 (threshold: 0.52) ✓
  A0301: 0.8676 (threshold: 0.4) ✓
  A2402: 0.0192 (threshold: 0.55) ✗
Peptide: VFLVLLPLV
  A0101: 0.0171 (threshold: 0.5) ✗
  A0201: 0.3831 (threshold: 0.45) ✗
  A0203: 0.0054 (threshold: 0.5) ✗
  A0207: 0.2326 (threshold: 0.52) ✗
  A0301: 0.0107 (threshold: 0.4) ✗
  A2402: 0.8655 (threshold: 0.55) ✓
Peptide: FLVLLPLVS
  A0101: 0.8592 (threshold: 0.5) ✓
  A0201: 0.9780 (threshold: 0.45) ✓
  A0203: 0.9939 (threshold: 0.5) ✓
  A0207: 0.8940 (threshold: 